# Python 迭代器与生成器 教学笔记
## 本章目标
- 理解可迭代对象、迭代器的概念与区别
- 掌握迭代器的手动实现与底层原理
- 理解生成器的核心特性与使用场景
- 掌握生成器函数、生成器表达式的使用
- 了解迭代器/生成器的内存优势

## 1. 初识迭代器

### 1.1 可迭代对象（Iterable）
能被 `for` 循环遍历的对象，核心特征：拥有 `__iter__()` 方法

In [1]:
# 测试哪些对象是可迭代对象
names = ['张三', '李四', '王五']  # 列表
citys = ('北京', '上海', '深圳')  # 元组
msg = 'hello'                    # 字符串
age = 10                         # 数字
def test(): pass                 # 函数

# 判断是否拥有 __iter__ 方法
print(f'列表: {hasattr(names, "__iter__")}')
print(f'元组: {hasattr(citys, "__iter__")}')
print(f'字符串: {hasattr(msg, "__iter__")}')
print(f'数字: {hasattr(age, "__iter__")}')
print(f'函数: {hasattr(test, "__iter__")}')

列表: True
元组: True
字符串: True
数字: False
函数: False


### 1.2 迭代器（Iterator）
调用可迭代对象的 `__iter__()` 方法（或 `iter()` 函数）得到的对象，核心特征：
- 拥有 `__next__()` 方法，每次调用返回下一个元素
- 遍历完后调用 `__next__()` 会抛出 `StopIteration` 异常
- 迭代器的 `__iter__()` 方法返回自身

In [2]:
# 获取迭代器并取值
names = ['张三', '李四', '王五']
it = iter(names)  # 等价于 names.__iter__()

# 逐个取值
print(next(it))   # 等价于 it.__next__()
print(next(it))
print(next(it))

# 遍历完后抛出异常（取消注释测试）
# print(next(it))

张三
李四
王五


### 1.3 for 循环的底层逻辑
for 循环本质是：获取迭代器 → 循环调用 next() → 捕获 StopIteration 异常

In [3]:
# 手动模拟 for 循环逻辑
names = ['张三', '李四', '王五']

# 1. 获取迭代器
it = iter(names)

# 2. 无限循环 + 异常捕获
while True:
    try:
        item = next(it)
        print(item)
    except StopIteration:
        break

张三
李四
王五


## 2. 手写迭代器

### 2.1 迭代器的特性
- 一次性：遍历过程中会被“消耗”，状态向前推进不重置
- 需实现 `__iter__()` 和 `__next__()` 方法（迭代器协议）

In [4]:
# 测试迭代器的一次性特性
names = ['张三', '李四', '王五']
it1 = iter(names)
it2 = iter(names)  # 新的迭代器

# 消耗 it1
print("it1 取值:")
print(next(it1))
print(next(it1))
print(next(it1))

# it1 已被消耗，再次遍历无输出
print("it1 遍历:")
for item in it1:
    print(item)

# it2 未被消耗，正常遍历
print("it2 遍历:")
for item in it2:
    print(item)

it1 取值:
张三
李四
王五
it1 遍历:
it2 遍历:
张三
李四
王五


### 2.2 自定义可迭代对象（Person类）
实现方式1：拆分可迭代对象和迭代器（推荐，职责分离）

In [5]:
class Person:
    def __init__(self, name, age, gender, address):
        self.name = name
        self.age = age
        self.gender = gender
        self.address = address
    
    def __iter__(self):
        # 返回迭代器实例
        return PersonIterator(self)

class PersonIterator:
    def __init__(self, p):
        self.p = p
        self.index = 0  # 迭代指针
        self.attrs = [p.name, p.age, p.gender, p.address]
    
    def __iter__(self):
        return self  # 迭代器返回自身
    
    def __next__(self):
        if self.index >= len(self.attrs):
            raise StopIteration
        # 获取当前值并推进指针
        value = self.attrs[self.index]
        self.index += 1
        return value

# 测试
p1 = Person('张三', 18, '男', '北京昌平')
for item in p1:
    print(item)

张三
18
男
北京昌平


### 2.3 简化实现：既是可迭代对象也是迭代器

In [6]:
# 进阶：迭代过程中修改值（字符串大写、数字转中文）

class Person:
    def __init__(self, name, age, gender, address):
        self.name = name
        self.age = age
        self.gender = gender
        self.address = address
        self.__index = 0  # 私有指针
        self.__attrs = [name, age, gender, address]
    
    def __iter__(self):
        self.__index = 0  # 每次迭代重置指针
        return self
    
    def __next__(self):
        if self.__index >= len(self.__attrs):
            raise StopIteration
        
        value = self.__attrs[self.__index]
        # 字符串转大写
        if isinstance(value, str):
            value = value.upper()
        
        self.__index += 1
        return value

# 测试
p1 = Person('zhangsan', 18, '男', '北京昌平')
for item in p1:
    print(item)

ZHANGSAN
18
男
北京昌平


## 3. 迭代器的优势

### 3.1 惰性计算，降低内存占用
迭代器不会一次性生成所有数据，而是按需生成，适合大数据量场景

In [7]:
import tracemalloc

# 迭代器实现斐波那契
class Fibo:
    def __init__(self, total):
        self.total = total
        self.index = 0
        self.pre = 1
        self.cur = 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.total:
            raise StopIteration
        if self.index < 2:
            value = 1
        else:
            value = self.pre + self.cur
            self.pre = self.cur
            self.cur = value
        self.index += 1
        return value

# 普通列表实现斐波那契
def fibo(total):
    if total <= 0:
        return []
    if total == 1:
        return [1]
    nums = [1, 1]
    for i in range(2, total):
        nums.append(nums[-1] + nums[-2])
    return nums

# 测试内存占用（取消注释运行，注意：10万条数据可能耗时）
# tracemalloc.start()
# f_iter = Fibo(100000)  # 迭代器版
# mem_iter = tracemalloc.get_traced_memory()[1]
# print(f'迭代器内存占用: {mem_iter / 1024 / 1024:.2f} MB')
# tracemalloc.stop()

# tracemalloc.start()
# f_list = fibo(100000)  # 列表版
# mem_list = tracemalloc.get_traced_memory()[1]
# print(f'列表内存占用: {mem_list / 1024 / 1024:.2f} MB')
# tracemalloc.stop()

# 按需取值示例
f1 = Fibo(100000)
for n in f1:
    if n > 100:
        break
    print(n)

1
1
2
3
5
8
13
21
34
55
89


## 4. 生成器（Generator）

### 4.1 生成器函数
函数体包含 `yield` 关键字的函数，调用后返回生成器对象（特殊的迭代器）
核心特性：
- 调用生成器函数不立即执行代码，而是返回生成器对象
- 调用 `next()` 或 `send()` 时执行代码，遇到 `yield` 暂停并返回值
- 再次调用从暂停位置继续执行

In [8]:
# 基础生成器函数示例
def demo():
    print('demo函数开始执行了')
    print(100)
    yield '我是第1个yield所返回的数据'
    a = 200
    print(a)
    yield '我是第2个yield所返回的数据'
    b = 300
    print(b)
    return '尚硅谷'

# 获取生成器对象（函数体未执行）
d = demo()
print(f'生成器对象: {d}')

# 第一次调用next()
r1 = next(d)
print(f'返回值1: {r1}')

# 第二次调用next()
r2 = next(d)
print(f'返回值2: {r2}')

# 第三次调用（触发return，抛出异常）
try:
    next(d)
except StopIteration as e:
    print(f'异常信息: {e}')

生成器对象: <generator object demo at 0x0000019AF82EC860>
demo函数开始执行了
100
返回值1: 我是第1个yield所返回的数据
200
返回值2: 我是第2个yield所返回的数据
300
异常信息: 尚硅谷


### 4.2 生成器的常用场景

In [9]:
# 场景1：循环中使用yield（批量生成数据）
def create_car(total):
    for index in range(1, total + 1):
        yield f'我是第{index}台车'

# 获取生成器对象
cars = create_car(5)

# 逐个取值
print(next(cars))
print(next(cars))
# 剩余值用for循环遍历
print("剩余车辆:")
for car in cars:
    print(car)

我是第1台车
我是第2台车
剩余车辆:
我是第3台车
我是第4台车
我是第5台车


In [10]:
# 场景2：yield from（批量yield可迭代对象）
def demo():
    nums = [10, 20, 30, 40]
    yield from nums  # 等价于 for n in nums: yield n

d = demo()
print("yield from 取值:")
for item in d:
    print(item)

yield from 取值:
10
20
30
40


In [11]:
# 场景3：send() 给yield传值
def demo():
    print('demo函数开始执行了')
    print(100)
    a = yield '我是第1个yield所返回的数据'
    print(f'收到传值: {a}')
    b = yield '我是第2个yield所返回的数据'
    print(f'收到传值: {b}')
    return '尚硅谷'

d = demo()
# 第一次启动必须传None
r1 = d.send(None)
print(f'返回值1: {r1}')

# 传值并获取下一个返回值
r2 = d.send(666)
print(f'返回值2: {r2}')

try:
    d.send(888)
except StopIteration as e:
    print(f'异常信息: {e}')

demo函数开始执行了
100
返回值1: 我是第1个yield所返回的数据
收到传值: 666
返回值2: 我是第2个yield所返回的数据
收到传值: 888
异常信息: 尚硅谷


### 4.3 生成器实现斐波那契数列

In [12]:
def fibo(total):
    pre = 1
    cur = 1
    for index in range(total):
        if index < 2:
            yield 1
        else:
            value = pre + cur
            pre = cur
            cur = value
            yield value

# 生成10个斐波那契数
f1 = fibo(10)
print("斐波那契数列前10项:")
for item in f1:
    print(item)

斐波那契数列前10项:
1
1
2
3
5
8
13
21
34
55


### 4.4 生成器表达式
语法：`(表达式 for 变量 in 可迭代对象)`，轻量级创建生成器的方式

In [13]:
# 对比列表推导式和生成器表达式
nums = [10, 20, 30, 40]

# 列表推导式（一次性生成所有数据）
list_result = [n * 2 for n in nums]
print(f'列表推导式: {list_result}')

# 生成器表达式（按需生成）
gen_result = (n * 2 for n in nums)
print(f'生成器对象: {gen_result}')
print(f'生成器取值: {list(gen_result)}')  # 转为列表（消耗生成器）

列表推导式: [20, 40, 60, 80]
生成器对象: <generator object <genexpr> at 0x0000019AF8353780>
生成器取值: [20, 40, 60, 80]


## 5. 总结
1. **可迭代对象**：拥有 `__iter__()` 方法，能被for遍历（列表、元组、字符串等）
2. **迭代器**：实现 `__iter__()` + `__next__()`，惰性取值，一次性消耗
3. **生成器**：特殊的迭代器，通过 `yield` 简化迭代器实现，更低的内存占用
4. **核心优势**：迭代器/生成器按需生成数据，适合大数据量、无限序列场景
5. **常用语法**：
   - 生成器函数：`def 函数(): yield 数据`
   - 生成器表达式：`(表达式 for 变量 in 可迭代对象)`
   - `yield from`：批量yield可迭代对象
   - `send()`：给yield传值（需先send(None)启动）